# <center><font color = 'blue'> The eight queens puzzle

The eight queens puzzle is the problem of placing eight chess queens on an 8x8
chessboard so that no two queens threaten each other. Thus, a solution requires
that no two queens share the same row, column, or diagonal.

The eight queens puzzle is an example of the more general n-queens problem of
placing n queens on an nxn chessboard, where solutions exist for all natural
numbers n with the exception of n=2 and n=3.

## References
* https://en.wikipedia.org/wiki/Eight_queens_puzzle

In [ ]:
try:
    import docplex.cp
except:
    !pip install docplex

In [ ]:
from docplex.cp.model import *
from sys import stdout

## Create model

Set model parameter

In [ ]:
NB_QUEEN = 8

Create CPO model

In [ ]:
mdl = CpoModel()

In [ ]:
# Create column index of each queen
x = integer_var_list(NB_QUEEN, 0, NB_QUEEN - 1, "X")

# One queen per raw
mdl.add(all_diff(x))

# One queen per diagonal xi - xj != i - j
mdl.add(all_diff(x[i] + i for i in range(NB_QUEEN)))

# One queen per diagonal xi - xj != j - i
mdl.add(all_diff(x[i] - i for i in range(NB_QUEEN)))

## Solve model

__Set your DOcloud credentials:__
0. A first option is to set the DOcloud url and key directly in the model source file *(see below)*
1. For a persistent setting, create a Python file __docloud_config.py__ somewhere that is visible from the __PYTHONPATH__

In [ ]:
print("\nSolving model....")
msol = mdl.solve(url="https://api-oaas.docloud.ibmcloud.com/job_manager/rest/v1/", 
                 key="ENTER YOUR KEY HERE")

## Display solution

Import required external libraries (numpy and matplotlib)

In [ ]:
try:
    import numpy as np
    import matplotlib.pyplot as plt
    VISU_ENABLED = True
except ImportError:
    VISU_ENABLED = False

In [ ]:
def display(sol):
    %matplotlib inline
    
    chess_board = np.zeros((NB_QUEEN, NB_QUEEN, 3))
    black = 0.5
    white = 1
    for l in range(8):
        for c in range(8):
            if (l%2 == c%2):
                col = white
            else:
                col = black
            chess_board[l,c,::]=col

    fig, ax = plt.subplots()
    ax.imshow(chess_board, interpolation='none')
    wq_im_file = "./n_queen_utils/WQueen.png"
    bq_im_file = "./n_queen_utils/BQueen.png"
    wq = plt.imread(wq_im_file)
    bq = plt.imread(bq_im_file)
    for y, x in enumerate(sol):
        if (x%2 == y%2):
            queen = bq
        else:
            queen = wq 
        ax.imshow(queen, extent=[x-0.4, x+0.4, y-0.4, y+0.4])
    ax.set(xticks=[], yticks=[])
    ax.axis('image')
    plt.show()

In [ ]:
if msol: 
    stdout.write("Solution:")
    sol = [msol[v] for v in x]
    for v in range(NB_QUEEN):
        stdout.write(" " + str(sol[v]))
    stdout.write("\n")
    stdout.write("Solve time: " + str(msol.get_solve_time()) + "\n")
    if VISU_ENABLED:
        display(sol)
else:
    stdout.write("No solution found\n")